# Machine-learning the surface-layer solver
*Chiel van Heerwaarden, 2020 (chiel.vanheerwaarden@wur.nl)*

$$
f_m \equiv \dfrac{ \kappa }
{ \ln \left( \dfrac{z}{z_{0m}} \right) 
- \Psi_m \left( \dfrac{z}{L} \right) 
+ \Psi_m \left( \dfrac{z_{0m}}{L} \right) }
$$

$$
f_h \equiv \dfrac{ \kappa }
{ \ln \left( \dfrac{z}{z_{0h}} \right) 
- \Psi_h \left( \dfrac{z}{L} \right) 
+ \Psi_h \left( \dfrac{z_{0h}}{L} \right) }
$$

$$
u_* = f_m \left( u_1 - u_0 \right)
$$

$$
\theta_{v*} = f_h \left( \theta_{v1} - \theta_{v0} \right)
$$

$$
L \equiv - \dfrac{u_*^3}{\kappa \dfrac{g}{\theta_{v00}} \overline{w^\prime \theta_v^\prime}_0}
$$

$$
Ri_B \equiv \dfrac{z g \left( \theta_{v1} - \theta_{v0} \right)}
{ \theta_{v00} \left( u_1 - u_0 \right)^2 }
=
\dfrac{z}{L} \dfrac{f_m^2}{f_h}
$$

___
# Create the training data
Predict from set $\left( Ri_B, z, z_{0m}, z_{0h} \right)$ the values of $(u_*, \theta_{v*}, L)$ 

In [ ]:
import numpy as np
from numba import jit
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LeakyReLU

In [ ]:
# Generate the input data.
z = 10
z0m = 0.1
z0h = 0.01
RiB = np.linspace(-5, 0.5, 20000)

In [ ]:
@jit(nopython=True, nogil=True, fastmath=True)
def phim(zeta):
    if zeta <= 0:
        phim = (1. + 3.6*(abs(zeta))**(2./3.))**(-1./2.)
    else:
        phim = 1. + 4.8*zeta
    return phim

@jit(nopython=True, nogil=True, fastmath=True)
def phih(zeta):
    if zeta <= 0:
        phih = (1. + 7.9*(abs(zeta))**(2./3.))**(-1./2.)
    else:
        phih = 1. + 7.8*zeta
    return phih

@jit(nopython=True, nogil=True, fastmath=True)
def psim(zeta):
    if zeta <= 0:
        x     = (1. - 16. * zeta)**(0.25)
        psim  = np.pi / 2. - 2. * np.arctan(x) + np.log((1. + x)**2. * (1. + x**2.) / 8.)
    else:
        psim  = -2./3. * (zeta - 5./0.35) * np.exp(-0.35 * zeta) - zeta - (10./3.) / 0.35
    return psim

@jit(nopython=True, nogil=True, fastmath=True)
def psih(zeta):
    if zeta <= 0:
        x     = (1. - 16. * zeta)**(0.25)
        psih  = 2. * np.log( (1. + x*x) / 2.)
    else:
        psih  = -2./3. * (zeta - 5./0.35) * np.exp(-0.35 * zeta) - (1. + (2./3.) * zeta) ** (1.5) - (10./3.) / 0.35 + 1.
    return psih

@jit(nopython=True, nogil=True, fastmath=True)
def RiB_to_L(RiB, z, z0m, z0h):
    if RiB > 0.:
        L  = 1.
        L0 = 2.
    else:
        L  = -1.
        L0 = -2.

    while abs(L - L0) > 1e-8:
        L0      = L
        fx      = RiB - z/L * (np.log(z/z0h) - psih(z/L) + psih(z0h/L)) / (np.log(z/z0m) - psim(z/L) + psim(z0m/L))**2.
        Lstart  = L - 0.001*L
        Lend    = L + 0.001*L
        fxdif   = ( (-z/Lstart * (np.log(z/z0h) - psih(z/Lstart) + psih(z0h/Lstart)) / \
                                 (np.log(z/z0m) - psim(z/Lstart) + psim(z0m/Lstart))**2.) \
                  - (-z/Lend   * (np.log(z/z0h) - psih(z/Lend  ) + psih(z0h/Lend  )) / \
                                 (np.log(z/z0m) - psim(z/Lend  ) + psim(z0m/Lend  ))**2.) ) / (Lstart-Lend)
        L       = L - fx/fxdif

        if(abs(L) > 1e15):
            break

    return L

In [ ]:
L = np.empty(RiB.size)
for i in range(RiB.size):
    L[i] = RiB_to_L(RiB[i], z, z0m, z0h)
zL = z/L

In [ ]:
n_train = 2**20

# Uniformly distributed RiB values.
#x_train = 3.2*np.random.rand(n_train) - 2.1

# Normally distributed RiB values
# x_train = np.random.normal(loc=0., scale=3., size=n_train)
# x_train = np.delete(x_train, np.argwhere(x_train > 0.6))

# More weight on neutral
RiB_train = 10.**(8.*np.random.rand(n_train//2) - 7.)
RiB_train = np.concatenate((RiB_train, -1 * 10.**(8.*np.random.rand(n_train//2) - 7.) ))
#np.random.shuffle(RiB_train)
RiB_train = np.delete(RiB_train, np.argwhere(RiB_train > 0.6))

print('Calculating training data')
zL_train = np.empty_like(RiB_train)
for i in range(RiB_train.size):
    zL_train[i] = z/RiB_to_L(RiB_train[i], z, z0m, z0h)

In [ ]:
plt.figure(figsize=(10,4))
plt.subplot(121)
plt.hist(RiB_train, 50)
plt.xlabel('RiB')
plt.subplot(122)
plt.hist(zL_train, 50)
plt.xlabel('z/L')
plt.tight_layout()

In [ ]:
model = Sequential()
model.add(Dense(units=50, input_dim=1))
model.add(LeakyReLU(0.01))
# model.add(Dense(units=20))
# model.add(LeakyReLU(0.01))
# model.add(Dense(units=20))
# model.add(LeakyReLU(0.01))
model.add(Dense(units=1))

model.compile(
        loss='mean_squared_error',
        optimizer='adam')

model.fit(RiB_train, zL_train, epochs=10, batch_size=128, verbose=1)

___
# Run inference

In [ ]:
loss_and_metrics = model.evaluate(RiB, zL, batch_size=1000, verbose=0)
print('loss: {:.5e}'.format(loss_and_metrics))

In [ ]:
zL_inf = np.array(model.predict(RiB, batch_size=1)).reshape(RiB.shape)
phim_inf = np.empty_like(zL_inf)

for i in range(phim_inf.size):
    phim_inf[i] = phim(zL_inf[i])
    
phim_ref = np.empty_like(zL)
for i in range(phim_ref.size):
    phim_ref[i] = phim(zL[i])

In [ ]:
plt.figure(figsize=(12,4))
plt.subplot(121)
plt.plot(RiB, zL, 'k:')
plt.plot(RiB, zL_inf)
plt.xlabel('RiB')
plt.ylabel('z/L')
plt.subplot(122)
plt.plot(RiB, zL, 'k:')
plt.plot(RiB, zL_inf)
plt.xlabel('RiB')
plt.ylabel('z/L')
plt.xlim(-0.5, 0.2)
plt.ylim(-1., 1.)
plt.tight_layout()

In [ ]:
plt.figure(figsize=(12,4))
plt.subplot(131)
plt.plot(zL, phim_ref, 'k:')
plt.plot(zL_inf, phim_inf)
plt.xlabel('z/L')
plt.ylabel('phim')
plt.subplot(132)
plt.semilogx(-zL, phim_ref, 'k:')
plt.semilogx(-zL_inf, phim_inf)
plt.xlabel('-z/L')
plt.ylabel('phim')
plt.xlim(1e-4, 10)
plt.ylim(0, 1.1)
plt.subplot(133)
plt.semilogx(zL, phim_ref, 'k:')
plt.semilogx(zL_inf, phim_inf)
plt.xlabel('z/L')
plt.ylabel('phim')
plt.xlim(1e-4, 1)
plt.ylim(0, 8.)
plt.tight_layout()